In [95]:
import pandas as pd
from attr import dataclass



In [117]:
df=pd.read_json("data/product_batch_00052.json")

In [130]:
df.columns

Index(['id', 'name', 'url_key', 'price', 'description', 'images'], dtype='object')

#Image

In [119]:
import os
import pandas as pd

def listurl(raw):
    return [x['base_url'] for x in raw if 'base_url' in x]

df1["images"] = df1["images"].apply(lambda v: v if isinstance(v, list) else [])
df1['images']=df1['images'].apply(listurl)
df1['images']=df1['images'].apply(lambda x: list(dict.fromkeys(x)))


for file in os.listdir("data"):
    path=os.path.join("data/",file)
    df=pd.read_json(path)
    df["images"] = df["images"].apply(lambda v: v if isinstance(v, list) else [])
    df['images']=df['images'].apply(listurl)
    df['images']=df['images'].apply(lambda x: list(dict.fromkeys(x)))
    df.to_json(path,orient='records',force_ascii=False,indent=2)


In [105]:
import os
import pandas as pd

In [120]:
folder="data"
dfs=[]
for file in os.listdir(folder):
    path=os.path.join(folder,file)
    df=pd.read_json(path)
    dfs.append(df)
raw=pd.concat(dfs,ignore_index=True)

In [122]:
raw.shape[0]

198942

In [129]:
records=[i[1:] for i in raw.itertuples()]

In [154]:
import psycopg2
from config import load_config

def insert_product(records):
    #read folder file json tiki product
    folder="data"
    dfs=[]
    for file in os.listdir(folder):
        path=os.path.join(folder,file)
        df=pd.read_json(path)
        dfs.append(df)
    raw=pd.concat(dfs,ignore_index=True)
    #insert to database tiki_products
    sql="INSERT INTO tiki_product(id, name, url_key, price, description, images) VALUES (%s, %s, %s, %s, %s, %s)"
    records=[i[1:] for i in raw.itertuples()]
    config=load_config()
    try:
        with  psycopg2.connect(**config) as conn:
            with  conn.cursor() as cur:
                # execute the INSERT statement
                cur.executemany(sql, records)
            # commit the changes to the database
            conn.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)

if __name__ == '__main__':
    insert_product(records)